In [1]:
!python -V

Python 3.9.12


In [2]:
# import library 
import pandas as pd 
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [5]:
import mlflow

# Set the tracking URI to the same one used in your UI
mlflow.set_tracking_uri("http://127.0.0.1:5001")  

# Create or set the experiment
mlflow.set_experiment("nyc-taxi-experiment")

2025/05/04 23:37:31 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1746401851956, experiment_id='1', last_update_time=1746401851956, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

### Put all the proccessing into a function

In [6]:
def read_dataframe(filename):
    "Write a function to read and preprocessing data"
    # Read the dataset
    df_taxi = pd.read_parquet(filename)

    # Adjust dropoff & pickup to pandas datetime 
    df_taxi['lpep_pickup_datetime'] = pd.to_datetime(df_taxi.lpep_pickup_datetime)
    df_taxi['lpep_dropoff_datetime'] = pd.to_datetime(df_taxi.lpep_dropoff_datetime)
    
    # Calculate the duration (drop_off -  pick_up)
    df_taxi['duration'] = df_taxi.lpep_dropoff_datetime - df_taxi.lpep_pickup_datetime
    
    # Adjust the duration in minutes for prediction 
    df_taxi['duration_minutes'] = df_taxi['duration'].dt.total_seconds() / 60

    # Since there are a lot of duration less than 1 minutes. We filter only duration between 1 minutes to 99% percentile
    df_taxi = df_taxi[(df_taxi['duration_minutes'] >= 1) & (df_taxi['duration_minutes'] <= 60)]

    # Feature Engineering 
    categorical_variables = ['PULocationID', 'DOLocationID']
    numerical_variables = ['trip_distance']

    # Convert it into "str"
    df_taxi[categorical_variables] = df_taxi[categorical_variables].astype(str)
    
    return df_taxi

In [7]:
df_train = read_dataframe('../00-Dataset/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../00-Dataset/green_tripdata_2021-02.parquet')

In [8]:
len(df_train), len(df_val)

(73908, 61921)

### Create the training pipeline 

In [9]:
# Feature Engineering 
categorical_variables = ['PULocationID', 'DOLocationID']
numerical_variables = ['trip_distance']

# Vectorizer the training variables 
dv = DictVectorizer()

# Convert it into dictionary 
train_dicts = df_train[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

# Create the validation set 
val_dicts = df_val[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [10]:
# Setup the Prediction_Variables 
predictor = 'duration_minutes'
y_train = df_train[predictor].values
y_val = df_val[predictor].values

In [11]:
# Train the model 
lr = LinearRegression() 
lr.fit(X_train, y_train)

# Make a prediction 
y_pred = lr.predict(X_val)

# Calculate the performance (RMSE)
mean_squared_error(y_val, y_pred, squared = False) 

10.47390313604031

### Try To Combine the input features 

In [12]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [13]:
# Feature Engineering 
categorical_variables = ['PU_DO']  #['PULocationID', 'DOLocationID']
numerical_variables = ['trip_distance']

# Vectorizer the training variables 
dv = DictVectorizer()

# Convert it into dictionary 
train_dicts = df_train[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

# Create the validation set 
val_dicts = df_val[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [14]:
# Setup the Prediction_Variables 
predictor = 'duration_minutes'
y_train = df_train[predictor].values
y_val = df_val[predictor].values

In [15]:
# Train the model 
lr = LinearRegression() 
lr.fit(X_train, y_train)

# Make a prediction 
y_pred = lr.predict(X_val)

# Calculate the performance (RMSE)
mean_squared_error(y_val, y_pred, squared = False) 

7.479586896299878

### Try with Lasso Regression

In [74]:
# Train the model 
lr = Lasso(alpha= 0.0001) 
lr.fit(X_train, y_train)

# Make a prediction 
y_pred = lr.predict(X_val)

# Calculate the performance (RMSE)
mean_squared_error(y_val, y_pred, squared = False) 

10.47054920225702

### Using MLFlow to automated 

In [16]:
with mlflow.start_run():
    
    #set the tag name for who response
    mlflow.set_tag("developer","dario")
    
    #set the training and validation
    mlflow.log_param("train-data-path", "../00-Dataset/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../00-Dataset/green_tripdata_2021-02.parquet")
    
    #set the alpha
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    
    
    lr = Lasso(alpha= 0.1) 
    lr.fit(X_train, y_train)

    # Make a prediction 
    y_pred = lr.predict(X_val)

    # Calculate the performance (RMSE)
    rmse = mean_squared_error(y_val, y_pred, squared = False) 
    mlflow.log_metric("rmse", rmse)

🏃 View run capricious-trout-752 at: http://127.0.0.1:5001/#/experiments/1/runs/dfc2a88bfc6b4ceb88fa586e9d6652f2
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/1
